# Coursera Capstone

### This notebook will be used for the Coursera Capstone Project - Segmenting and Clustering Neighborhoods in Toronto.
#### _$Author: Diego Assis$_

# PART 1

In [2]:
!conda install -c conda-forge lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.3.4                 |   py36h7ec2d77_0         1.5 MB  conda-forge

The following packages will be UPDATED:

    lxml: 4.2.5-py37hefd8a0e_0 --> 4.3.4-py36h7ec2d77_0 conda-forge


lxml-4.3.4           | 1.5 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
!conda install -c conda-forge geocoder --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    click-7.0                  |             py_0          61 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    future-0.17.1              |        py36_1000         701 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         821 KB

The following NEW packages will be INSTALLED:

    future:   0.17.1-py36_1000 co

In [4]:
!conda install -c conda-forge geopy --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

In [5]:
import json
import numpy as np
import urllib.request
from urllib.request import urlopen
import matplotlib.cm as cm
import matplotlib.colors as colors
import random
import pandas as pd
import geocoder
import geopy
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans

In [6]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [7]:
tables = pd.read_html(url)
tables = tables[0]
tables.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### *Creating a table using Wikipedia page*
===================================

In [8]:
column_names = ['PostalCode', 'Borough', 'Neighborhood']
TorontoTB = pd.DataFrame(columns=column_names)
TorontoTB

,PostalCode,Borough,Neighborhood


### *Creating a blank table with the 3 headers asked*
===========================================

In [9]:
for index, row in tables.iterrows():
    postal_code = row['Postcode']
    borough = row['Borough']
    neighborhood_name = row['Neighbourhood']
    
    if row['Borough'] != 'Not assigned':
        TorontoTB = TorontoTB.append({'PostalCode': postal_code,
                                      'Borough': borough,
                                      'Neighborhood': neighborhood_name}, ignore_index=True)

TorontoTB.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### *Adding the values coming from the wikipedia to the Toronto's Table*
### *At this time I am ignoring Borough with "Not assigned" value as per requirement*
==============================================================

In [10]:
def f(x):
     return pd.Series(dict(PostalCode = x['PostalCode'].unique(), 
                        Borough = x['Borough'].unique(), 
                        Neighborhood = "{%s}" % ', '.join(x['Neighborhood'])))

TorontoTB2 = TorontoTB.groupby('PostalCode').apply(f)
TorontoTB2 = TorontoTB2.reset_index(drop=True)

### *Here I am grouping the table by PostalCode*
### *The I clean the data and convert the list and tuples back to Str*
=========================================

In [11]:
#TorontoSeries.dtypes
TorontoTB2.head()

,PostalCode,Borough,Neighborhood
0,[M1B],[Scarborough],"{Rouge, Malvern}"
1,[M1C],[Scarborough],"{Highland Creek, Rouge Hill, Port Union}"
2,[M1E],[Scarborough],"{Guildwood, Morningside, West Hill}"
3,[M1G],[Scarborough],{Woburn}
4,[M1H],[Scarborough],{Cedarbrae}


In [12]:
TorontoTB2['PostalCode'] = TorontoTB2['PostalCode'].str[0]
TorontoTB2['Borough'] = TorontoTB2['Borough'].str[0]
TorontoTB2['Neighborhood'] = TorontoTB2['Neighborhood'].str.strip('{}')
TorontoTB2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
for index, row in TorontoTB2.iterrows():
    if row.Neighborhood == 'Not assigned':
        print("OPQ")
        print(row.PostalCode, row.Borough, row.Neighborhood)
        row.Neighborhood = row.Borough

OPQ
M7A Queen's Park Not assigned


### *In the end I just make sure that any "Not assigned" Neighborhood will receive the Borough value as per requirements*
=========================================

In [14]:
TorontoTB2.shape

(103, 3)

# PART 2

### *Google API is instable so I will work on the Geospatial CSV given by coursera*
=========================================

In [15]:
filename = "http://cocl.us/Geospatial_data"
geo_pd = pd.read_csv(filename)

In [16]:
geo_pd.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
TorontoTB2['Latitude'] = 0
TorontoTB2['Longitude'] = 0
TorontoTB2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",0,0
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",0,0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",0,0
3,M1G,Scarborough,Woburn,0,0
4,M1H,Scarborough,Cedarbrae,0,0


### *Just adding 2 last columns "Latitude" and "Longitude"*
=========================================

In [18]:
for index, row in geo_pd.iterrows():
    ptcd = row['Postal Code']
    lat = row['Latitude']
    lng = row['Longitude']

    TorontoTB2.loc[TorontoTB2['PostalCode'] == ptcd, ['Latitude']] = lat
    TorontoTB2.loc[TorontoTB2['PostalCode'] == ptcd, ['Longitude']] = lng

TorontoTB2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### *Using the CSV to populate my table with correct Latitude and Longitude by Postal Code*
=========================================

In [19]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


# PART 3

### *Using geopy library to get the latitude and longitude values of Toronto City and generate a Folium Map*
=========================================

In [20]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(TorontoTB2['Latitude'], TorontoTB2['Longitude'], TorontoTB2['Borough'], TorontoTB2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### *I decided to use Downtown Borough due to a good quantity of neighborhood*
=========================================

In [21]:
TorontoTB2['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [22]:
downtown_data = TorontoTB2[TorontoTB2['Borough'] == "Downtown Toronto"].reset_index(drop=True)
downtown_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


### *Generate a Folium Map of Downtown Toronto and marking the neighborhood*
=========================================

In [23]:
address = 'Downtown Toronto, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.65145205, -79.3620078930752.


In [24]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=12.5)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown